In [3]:
import re, json, pandas as pd
from pathlib import Path

In [18]:
log_path = Path("../logs/autorizador/autorizador.log")

events = []

with log_path.open(encoding="utf-8", errors="ignore") as f:
    lines = f.readlines()

current_event = {}
for ln in lines:
    # Detectar request a validate-signature
    if "Incoming request: POST /validate-signature" in ln:
        current_event = {"test1_firma_valida": None, "test2_tampered_payload": False, "test3_tampered_firma": None, "case": None}

    if "Payload:" in ln and current_event:
        txt = ln.split("Payload:",1)[1].strip()
        safe = (txt.replace("'", '"')
                   .replace("True","true")
                   .replace("False","false")
                   .replace("None","null"))
        try:
            payload = json.loads(safe)
        except Exception:
            payload = {}
        direccion = (payload.get("payload") or {}).get("direccion","")
        if "ALTERADA" in direccion.upper():
            current_event["test2_tampered_payload"] = True

    if "Response JSON:" in ln and current_event:
        txt = ln.split("Response JSON:",1)[1].strip()
        safe = (txt.replace("'", '"')
                   .replace("True","true")
                   .replace("False","false")
                   .replace("None","null"))
        try:
            resp = json.loads(safe)
        except Exception:
            resp = {}
        current_event["test1_firma_valida"] = resp.get("firma_valida")

        # Clasificar caso
        if current_event["test1_firma_valida"]:
            current_event["case"] = "firma_valida"
            expected = True
        else:
            if current_event["test2_tampered_payload"]:
                current_event["case"] = "tampered_payload"
            else:
                current_event["case"] = "tampered_firma"
                current_event["test3_tampered_firma"] = True
            expected = False

        current_event["test_ok"] = (current_event["test1_firma_valida"] == expected)
        events.append(current_event)
        current_event = {}

In [19]:
# DataFrame con resultados
df = pd.DataFrame(events)
display(df)

,test1_firma_valida,test2_tampered_payload,test3_tampered_firma,case,test_ok
0,True,False,None,firma_valida,True
1,False,True,None,tampered_payload,True
2,False,False,True,tampered_firma,True
3,True,False,None,firma_valida,True
4,True,False,None,firma_valida,True
...,...,...,...,...,...
102,False,False,True,tampered_firma,True
103,True,False,None,firma_valida,True
104,True,False,None,firma_valida,True
105,True,False,None,firma_valida,True


In [20]:
# Resumen por caso
summary = (
    df.groupby("case")["test_ok"]
      .agg(total="count", passed="sum")
      .reset_index()
)
summary["pass_rate_%"] = (summary["passed"] / summary["total"] * 100).round(2)
display(summary)

,case,total,passed,pass_rate_%
0,firma_valida,53,53,100.0
1,tampered_firma,27,27,100.0
2,tampered_payload,27,27,100.0


In [21]:
# Resumen global
overall = {
    "total_tests": len(df),
    "passed": int(df["test_ok"].sum()),
    "overall_pass_rate_%": round(df["test_ok"].mean()*100,2) if not df.empty else 0
}
print(overall)

{'total_tests': 107, 'passed': 107, 'overall_pass_rate_%': np.float64(100.0)}
